In [ ]:
# Something about data.
import os

index_dir = os.path.join("index")
index_test_path = os.path.join(index_dir, "gene_index.01_test.csv")
index_training_validation_path = os.path.join(index_dir, "gene_index.01_train_validation.csv")
gene_dir = os.path.join("data", "gene_dir")



In [8]:
# Import Lib

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import time
import tensorflow as tf
import keras.utils
from keras.layers import Embedding, Dense, Flatten, Dropout, SpatialDropout1D, TimeDistributed, LSTM, GRU, Bidirectional
from keras.models import Sequential
from keras.layers.convolutional import Conv1D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from keras import Input, Model
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from collections import Counter
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.model_selection import KFold
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
import os

baseline_dir = os.path.join("data", "baseline")
model_dir = os.path.join(baseline_dir, "model")
data_dir = os.path.join(baseline_dir, "kmer")
num_classes = 8

In [7]:
import numpy as np
characters = ["A", "C", "G", "T"]

kmer_dict = {}
count = 0
embedding_vectors = []
for a in characters:
  for b in characters:
    for c in characters:
      v = [0 for i in range(64)]
      kmer_dict[f"{a}{b}{c}"] = count
      v[count] = 1
      embedding_vectors.append(v)
      count += 1

embedding_vectors = np.array(embedding_vectors)
print(embedding_vectors.shape)
print(kmer_dict)
embedding_matrix = embedding_vectors

Label_Dictionary = {
    '[CLS]': -100, #_create_one_hot_encoding(0, 10),
    '[SEP]': -100,
    # '[PAD]': 2, #_create_one_hot_encoding(9, 10)
    'III': -100,   # Created III instead of iii for PAD special token. 
                # This is for enabling reading contigs if token is predicted as padding. 
                # #_create_one_hot_encoding(9, 10)
    'iii': 0,   #_create_one_hot_encoding(1, 10),
    'iiE': 1,   #_create_one_hot_encoding(2, 10),
    'iEi': 2,   #_create_one_hot_encoding(3, 10),
    'Eii': 3,   #_create_one_hot_encoding(4, 10),
    'iEE': 4,   #_create_one_hot_encoding(5, 10),
    'EEi': 5,   #_create_one_hot_encoding(6, 10),
    'EiE': 6,   #_create_one_hot_encoding(7, 10),
    'EEE': 7,  #_create_one_hot_encoding(8, 10),
}

label_dict = Label_Dictionary

Index_Dictionary = {
    #0: "[CLS]",
    #1: "[SEP]",
    #2: "III",       # Use `III` as padding symbol.
    0: "iii",
    1: "iiE",
    2: "iEi",
    3: "Eii",
    4: "iEE",
    5: "EEi",
    6: "EiE",
    7: "EEE",
    -100: "[CLS]/[SEP]/[III]"
}

import os
import pandas as pd
import numpy as np
from tqdm import tqdm

training_data_path = os.path.join(data_dir, "gene_index.01_train_validation_ss_all_pos_train.csv")
validation_data_path = os.path.join(data_dir, "gene_index.01_train_validation_ss_all_pos_validation.csv")
test_data_path = os.path.join(data_dir, "gene_index.01_test_ss_all_pos.csv")

def preprocessing(data_path):
  if os.path.exists(data_path):
    print(f"data exists at {data_path}")
  else:
    raise FileNotFoundError(f"data not found ata {data_path}")
  encoded_sequences = []
  encoded_labels = []
  df = pd.read_csv(data_path)
  for i, r in tqdm(df.iterrows(), total=df.shape[0], desc="Preprocessing"):
    sequence = r["sequence"]
    label = r["label"]

    encoded_sequence = [kmer_dict[a] for a in sequence.split(' ')]
    encoded_label = [label_dict[a] for a in label.split(' ')]

    if len(encoded_sequence) != 512 or len(encoded_label) != 512:
      raise ValueError(f"input size not correct. expected 512 and 512, found {len(encoded_sequence)} {len(encoded_label)}")

    encoded_sequences.append(
        encoded_sequence
    )
    encoded_labels.append(
        encoded_label
    )
  return encoded_sequences, encoded_labels


X_train, Y_train = preprocessing(training_data_path)
X_train = np.array(X_train)
Y_train = np.array([tf.keras.utils.to_categorical(y, num_classes=num_classes) for y in Y_train])
X_val, Y_val = preprocessing(validation_data_path)
X_val = np.array(X_val)
Y_val = np.array([tf.keras.utils.to_categorical(y, num_classes=num_classes) for y in Y_val])
X_test, Y_test = preprocessing(test_data_path)
X_test = np.array(X_test)
Y_test = np.array([tf.keras.utils.to_categorical(y, num_classes=num_classes) for y in Y_test])

print(f"Training data {np.array(X_train).shape}, {np.array(Y_train).shape}")
print(f"Validation data {np.array(X_val).shape}, {np.array(Y_val).shape}")
print(f"Test data {np.array(X_test).shape}, {np.array(Y_test).shape}")

(64, 64)
{'AAA': 0, 'AAC': 1, 'AAG': 2, 'AAT': 3, 'ACA': 4, 'ACC': 5, 'ACG': 6, 'ACT': 7, 'AGA': 8, 'AGC': 9, 'AGG': 10, 'AGT': 11, 'ATA': 12, 'ATC': 13, 'ATG': 14, 'ATT': 15, 'CAA': 16, 'CAC': 17, 'CAG': 18, 'CAT': 19, 'CCA': 20, 'CCC': 21, 'CCG': 22, 'CCT': 23, 'CGA': 24, 'CGC': 25, 'CGG': 26, 'CGT': 27, 'CTA': 28, 'CTC': 29, 'CTG': 30, 'CTT': 31, 'GAA': 32, 'GAC': 33, 'GAG': 34, 'GAT': 35, 'GCA': 36, 'GCC': 37, 'GCG': 38, 'GCT': 39, 'GGA': 40, 'GGC': 41, 'GGG': 42, 'GGT': 43, 'GTA': 44, 'GTC': 45, 'GTG': 46, 'GTT': 47, 'TAA': 48, 'TAC': 49, 'TAG': 50, 'TAT': 51, 'TCA': 52, 'TCC': 53, 'TCG': 54, 'TCT': 55, 'TGA': 56, 'TGC': 57, 'TGG': 58, 'TGT': 59, 'TTA': 60, 'TTC': 61, 'TTG': 62, 'TTT': 63}
data exists at data\baseline\kmer\gene_index.01_train_validation_ss_all_pos_train.csv


Preprocessing: 100%|██████████| 71058/71058 [00:14<00:00, 4878.63it/s]


data exists at data\baseline\kmer\gene_index.01_train_validation_ss_all_pos_validation.csv


Preprocessing: 100%|██████████| 17765/17765 [00:03<00:00, 4801.34it/s]


data exists at data\baseline\kmer\gene_index.01_test_ss_all_pos.csv


Preprocessing: 100%|██████████| 6977/6977 [00:01<00:00, 5044.95it/s]


Training data (71058, 512), (71058, 512, 8)
Validation data (17765, 512), (17765, 512, 8)
Test data (6977, 512), (6977, 512, 8)


In [34]:
bilstm_model = tf.keras.models.load_model(
    os.path.join(model_dir, "model_bilstm.h5")
    )

bigru_model = tf.keras.models.load_model(
    os.path.join(model_dir, "model_bigru.h5")
    )

for m in [bilstm_model, bigru_model]:

    print(X_test.shape, Y_test.shape)
    y_pred = m.predict(X_test)
    print(y_pred.shape)

    print(y_pred.shape, Y_test.shape)

    y_pred_ids = np.argmax(y_pred, axis=-1)
    y_test_ids = np.argmax(Y_test, axis=-1)

    print(y_pred_ids.shape, y_test_ids.shape)

    y_pred_ids_flatten = y_pred_ids.flatten()
    y_test_ids_flatten = y_test_ids.flatten()

    label_indices = [i for i in range(num_classes)]
    print(y_pred_ids_flatten.shape, y_test_ids_flatten.shape)
    print(y_pred_ids_flatten, all([a in label_indices for a in y_pred_ids_flatten]))
    print(y_test_ids_flatten, all([a in label_indices for a in y_test_ids_flatten]))
    print(num_classes)
    print(classification_report(
        y_test_ids_flatten, 
        y_pred_ids_flatten, 
        labels=[i for i in range(num_classes)], 
        target_names=[Index_Dictionary.get(i) for i in range(num_classes)], 
        zero_division=0))

(6977, 512) (6977, 512, 8)
219/219 [==============================] - 94s 421ms/step
(6977, 512, 8)
(6977, 512, 8) (6977, 512, 8)
(6977, 512) (6977, 512)
(3572224,) (3572224,)
[7 7 7 ... 7 7 7] True
[7 7 7 ... 7 7 7] True
8
              precision    recall  f1-score   support

         iii       0.82      0.87      0.85   2697709
         iiE       0.81      0.30      0.44      5857
         iEi       0.00      0.00      0.00         0
         Eii       0.61      0.36      0.45      5876
         iEE       0.75      0.26      0.38      5856
         EEi       0.58      0.36      0.45      5875
         EiE       0.00      0.00      0.00         0
         EEE       0.50      0.41      0.45    851051

   micro avg       0.76      0.76      0.76   3572224
   macro avg       0.51      0.32      0.38   3572224
weighted avg       0.74      0.76      0.75   3572224

(6977, 512) (6977, 512, 8)
219/219 [==============================] - 95s 427ms/step
(6977, 512, 8)
(6977, 512, 8) (6977, 512

In [33]:
from sklearn.metrics import classification_report
y_true = [0, 1, 2, 2, 2]
y_pred = [0, 0, 2, 2, 1]
target_names = ['class 0', 'class 1', 'class 2']
target_indices = [0, 1, 2, 3, 4]
print(classification_report(y_true, y_pred, labels=target_indices))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.00      0.00      0.00         1
           2       1.00      0.67      0.80         3
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0

   micro avg       0.60      0.60      0.60         5
   macro avg       0.30      0.33      0.29         5
weighted avg       0.70      0.60      0.61         5



c:\.virtualenv\sequence-processing-310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\.virtualenv\sequence-processing-310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\.virtualenv\sequence-processing-310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\.virtualenv\sequen

In [31]:
from sklearn.metrics import classification_report

label_names = ['class A', 'class B', 'class C', 'class D', 'class E', 'class F']
y_true = [1, 2, 3, 2, 5]
y_pred = [0, 2, 3, 4, 5]
print(classification_report(y_true, y_pred, labels=label_names))

c:\.virtualenv\sequence-processing-310\lib\site-packages\numpy\lib\arraysetops.py:608: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''